Business Requirement:

A Chinese restraunt Grandma’s Kitchen wants to open an office in New york (Manhattan).
This is project to recommend them a solution for branch setup.

Business Input/data

I will use the data from internet for this which is already available from earlier modules. 
https://geo.nyu.edu/catalog/nyu_2451_34572
For the data about Chinese restaurants in each neighborhoods I am going to use the four square data.


In [3]:
import pandas as pd
import requests

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import numpy as np # library to handle data in a vectorized manner
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


# import k-means from clustering stage
from sklearn.cluster import KMeans
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.17.0-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  24.73 MB/s
geopy-1.17.0-p 100% |################################| Time: 0:00:00  35.08 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  50.10 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  35.68 MB/s
vincent-0.4.4- 100% |###################

In [4]:
# Download and Load data 

# Download data from staged repository
!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('Data downloaded!')


Data downloaded!


In [5]:
# convert Data into JSON
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [6]:
# Pull the data into a new variable from the JSON data
neighborhoods_data = newyork_data['features']


In [7]:
#Create Dataframe columns and instantiate the dataframe
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=column_names)


In [8]:
#loop through the dataframe and fill the dataframe

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
neighborhoods.head()


,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


Find the Data for Manhattan 



In [9]:
neighborhoods = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
neighborhoods.shape

(40, 4)

Exploring the data set

In [10]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)


The dataframe has 1 boroughs and 40 neighborhoods.


get the latitude and longitude values of New York City.

In [11]:
address = 'Manhattan, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))


/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of New York City are 40.7900869, -73.9598295.


Create a map of New York with Markers of neighbourhood - Map not showing up in Git but it works in Jupter notebook

In [12]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork


In [13]:
#Exploring neighbourhood
neighborhoods.loc[0, 'Neighborhood']


'Marble Hill'

In [14]:
#Get the coordinates
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))


Latitude and longitude values of Marble Hill are 40.87655077879964, -73.91065965862981.


In [15]:
# The code was removed by Watson Studio for sharing.

In [16]:
#get the top venues that are in Marble Hill within a radius of hundred to five hudred meters. 
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
section = 'food' # find category of food from four squeare
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&section={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT,
    section)

results = requests.get(url).json()



In [17]:
# extract the category of the venue

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


clean the json and structure it into a pandas dataframe

In [18]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON


In [19]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

print('{} food venue(s) were returned by Foursquare for {}'.format(nearby_venues.shape[0], neighborhoods.loc[0, 'Neighborhood']) )


12 food venue(s) were returned by Foursquare for Marble Hill


Explore Neighborhoods in New York

In [20]:
def getNearbyVenues(names, latitudes, longitudes, section, radius=500):
    venues_list=[]
    i=1
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name,i)
        i = i + 1
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&section={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            section)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)



In [21]:
newyork_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude'],
                                   section='food')

print(newyork_venues.shape)
newyork_venues.head()


Marble Hill 1
Chinatown 2
Washington Heights 3
Inwood 4
Hamilton Heights 5
Manhattanville 6
Central Harlem 7
East Harlem 8
Upper East Side 9
Yorkville 10
Lenox Hill 11
Roosevelt Island 12
Upper West Side 13
Lincoln Square 14
Clinton 15
Midtown 16
Murray Hill 17
Chelsea 18
Greenwich Village 19
East Village 20
Lower East Side 21
Tribeca 22
Little Italy 23
Soho 24
West Village 25
Manhattan Valley 26
Morningside Heights 27
Gramercy 28
Battery Park City 29
Financial District 30
Carnegie Hill 31
Noho 32
Civic Center 33
Midtown South 34
Sutton Place 35
Turtle Bay 36
Tudor City 37
Stuyvesant Town 38
Flatiron 39
Hudson Yards 40
(2852, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
2,Marble Hill,40.876551,-73.91066,Dunkin' Donuts,40.876993,-73.906507,Donut Shop
3,Marble Hill,40.876551,-73.91066,Land & Sea Restaurant,40.877885,-73.905873,Seafood Restaurant
4,Marble Hill,40.876551,-73.91066,Parrilla Latina,40.877473,-73.906073,Steakhouse


In [22]:
newyork_venues.groupby('Neighborhood').size().reset_index(name='counts')

,Neighborhood,counts
0,Battery Park City,38
1,Carnegie Hill,69
2,Central Harlem,46
3,Chelsea,100
4,Chinatown,100
5,Civic Center,86
6,Clinton,100
7,East Harlem,50
8,East Village,100
9,Financial District,100


In [23]:
# one hot encoding
newyork_onehot = pd.get_dummies(newyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
newyork_onehot['Neighborhood'] = newyork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [newyork_onehot.columns[-1]] + list(newyork_onehot.columns[:-1])
newyork_onehot = newyork_onehot[fixed_columns]
newyork_onehot.head()

newyork_onehot.shape


(2852, 122)

In [24]:
# Group rows by neighborhood by taking the mean of the frequency of occurrence of each category

newyork_grouped = newyork_onehot.groupby('Neighborhood').mean().reset_index()
newyork_grouped.shape

newyork_grouped.shape


(40, 122)

In [25]:
#Let's put that into a pandas dataframe

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 4

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = newyork_grouped['Neighborhood']

for ind in np.arange(newyork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(newyork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

neighborhoods_venues_sorted.shape


(40, 5)

k-means Clustering

In [26]:
# set number of clusters
kclusters = 6

newyork_grouped_clustering = newyork_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(newyork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
newyork_grouped_clustering.shape

newyork_grouped_clustering


,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,...,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Tonkatsu Restaurant,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Wings Joint
0,0.00,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,...,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000
1,0.00,0.000000,0.014493,0.000000,0.014493,0.000000,0.000000,0.000000,0.000000,0.014493,...,0.028986,0.000000,0.00,0.00,0.014493,0.00,0.000000,0.00,0.028986,0.000000
2,0.00,0.065217,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,0.021739,...,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.021739,0.00,0.000000,0.000000
3,0.00,0.000000,0.050000,0.010000,0.000000,0.030000,0.010000,0.000000,0.010000,0.020000,...,0.020000,0.000000,0.00,0.00,0.000000,0.00,0.030000,0.00,0.030000,0.000000
4,0.00,0.000000,0.030000,0.000000,0.000000,0.020000,0.010000,0.010000,0.000000,0.010000,...,0.010000,0.000000,0.00,0.00,0.000000,0.00,0.030000,0.00,0.060000,0.000000
5,0.00,0.000000,0.069767,0.000000,0.000000,0.023256,0.011628,0.000000,0.000000,0.011628,...,0.011628,0.011628,0.00,0.00,0.000000,0.00,0.011628,0.00,0.011628,0.011628
6,0.00,0.000000,0.080000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,...,0.030000,0.000000,0.00,0.00,0.000000,0.00,0.020000,0.00,0.000000,0.000000
7,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.040000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000
8,0.00,0.000000,0.020000,0.020000,0.010000,0.010000,0.000000,0.000000,0.010000,0.020000,...,0.010000,0.000000,0.00,0.00,0.000000,0.01,0.040000,0.00,0.050000,0.010000
9,0.00,0.000000,0.070000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.010000,...,0.010000,0.000000,0.00,0.00,0.000000,0.00,0.030000,0.00,0.000000,0.000000


In [28]:
#create a new dataframe that includes the cluster as well as the top 10 venues
newyork_merged = neighborhoods
newyork_merged.shape
# add clustering labels
newyork_merged['Cluster Labels'] = kmeans.labels_

# merge newyork_grouped with newyork_data to add latitude/longitude for each neighborhood
newyork_merged = newyork_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

newyork_merged.head() # check the last columns!


,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,5,Sandwich Place,Deli / Bodega,Restaurant,American Restaurant
1,Manhattan,Chinatown,40.715618,-73.994279,5,Chinese Restaurant,Dim Sum Restaurant,Vietnamese Restaurant,Bakery
2,Manhattan,Washington Heights,40.851903,-73.936900,4,Deli / Bodega,Pizza Place,Bakery,Mexican Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,0,Deli / Bodega,Pizza Place,Mexican Restaurant,Restaurant
4,Manhattan,Hamilton Heights,40.823604,-73.949688,4,Deli / Bodega,Café,Mexican Restaurant,Donut Shop


In [29]:
# visualize the resulting clusters

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(newyork_merged['Latitude'], newyork_merged['Longitude'], newyork_merged['Neighborhood'], newyork_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


Examine the Cluster

In [31]:
#Cluster1-Red 

newyork_merged.loc[newyork_merged['Cluster Labels'] == 0, 
                   newyork_merged.columns[[1] + 
                   list(range(5, newyork_merged.shape[1]))]]

#Cluster2-Purple

newyork_merged.loc[newyork_merged['Cluster Labels'] == 1, 
                   newyork_merged.columns[[1] + 
                   list(range(5, newyork_merged.shape[1]))]]

#Cluster3-Blue

newyork_merged.loc[newyork_merged['Cluster Labels'] == 2, 
                   newyork_merged.columns[[1] + 
                   list(range(5, newyork_merged.shape[1]))]]

#Cluster4-green


newyork_merged.loc[newyork_merged['Cluster Labels'] == 3, 
                   newyork_merged.columns[[1] + 
                   list(range(5, newyork_merged.shape[1]))]]

#Cluster5-lightgreen

newyork_merged.loc[newyork_merged['Cluster Labels'] == 4, 
                   newyork_merged.columns[[1] + 
                   list(range(5, newyork_merged.shape[1]))]]

#Cluster6-orange

newyork_merged.loc[newyork_merged['Cluster Labels'] == 5, 
                   newyork_merged.columns[[1] + 
                   list(range(5, newyork_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
0,Marble Hill,Sandwich Place,Deli / Bodega,Restaurant,American Restaurant
1,Chinatown,Chinese Restaurant,Dim Sum Restaurant,Vietnamese Restaurant,Bakery
8,Upper East Side,Italian Restaurant,American Restaurant,Pizza Place,Diner
11,Roosevelt Island,Deli / Bodega,Sandwich Place,Bakery,Kosher Restaurant
12,Upper West Side,Italian Restaurant,Indian Restaurant,Vegetarian / Vegan Restaurant,French Restaurant
16,Murray Hill,Sandwich Place,American Restaurant,Japanese Restaurant,Pizza Place
20,Lower East Side,Pizza Place,Chinese Restaurant,Bakery,Japanese Restaurant
27,Gramercy,Italian Restaurant,Deli / Bodega,Restaurant,Mexican Restaurant
31,Noho,Italian Restaurant,Pizza Place,Mexican Restaurant,Sushi Restaurant
35,Turtle Bay,Italian Restaurant,Café,Deli / Bodega,Steakhouse


Solution:
Using the k means clustering Four square API calls got the necessary details requried  to suggest solution.

looking at the clusters here is what i suggest:
Cluster1
More Italian restaurants.
Cluster2
More Italian restaurants and deli shops
Cluster3
Chinese restaurant is the top choice.
Cluster4
American restaurant & Indian restaurant
Cluster5
wide spread restaurant options.
Cluster6
more pizza places.

Solution: 
We can have more focus on cluster three as grandma kitchen is chinese restaurant which suggest that would be a good area for business setup as it is mostly focused in two areas. One in the lower east side and another in the upper west side. Looking furthur into this area the lower east side with China town has more possibility for grandma' kitchen branch setup, evevnthough there is a bit of competetion.

Conclusion:
Based on the above results it would be a great suggestion for the grandma's kitchen business to setup their branch close to cluster 3 primarily close to China town and Lower East side. This business can later expand based on the demand and popularity to Midtown south neightbourhood which is also in the same cluster 3. Further it can also be analysed to see the number of restaurant and the revenue and profits from this location for this business location. The Grandma's kitchen should setup thier branch close to Lower East, China town, Midtown, south and Brooklyn.
